In [0]:
!pip3 install xgboost==1.0.2

import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
#import tensorflow
import random
import json

     |████████████████████████████████| 109.8MB 90kB/s 
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Load Data set

In [0]:
%cd '/content/drive/My Drive/sem5_group_project/'
with open("intents.json") as file:
  data=json.load(file)

/content/drive/My Drive/sem5_group_project


# Data Pre Processing


In [0]:
texts = []
tags = []
for intent in data["intents"]:
  tag=intent["tag"]
  for pattern in intent["patterns"]:
    texts.append(pattern)
    tags.append(tag)

In [0]:
import re
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()# lowercase text
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, "", text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text = " " + text + " "
    for sw in STOPWORDS:
        text = text.replace(" "+sw+" ", " ") # delete stopwords from text
    text = re.sub('[ ][ ]+', " ", text)
    if len(text)>1:
      if text[0] == ' ':
          text = text[1:]
      if text[-1] == ' ':
          text = text[:-1]
        
    return text

In [0]:
X_train = [text_prepare(text) for text in texts]
y_train = [text_prepare(tag) for tag in tags]

In [0]:
labels=sorted(set(y_train))

In [0]:
out_empty=[0 for _ in range(len(labels))]
output=[]

In [0]:
for tag in y_train:
  if tag=="bus complaints":
    output.append([1,0,0,0,0,0,0,0,0,0,0,0])
  if tag=="bus fees":
    output.append([0,1,0,0,0,0,0,0,0,0,0,0])
  if tag=="bus ticket booking":
    output.append([0,0,1,0,0,0,0,0,0,0,0,0])
  if tag=="bus times":
    output.append([0,0,0,1,0,0,0,0,0,0,0,0])
  if tag=="distance places":
    output.append([0,0,0,0,1,0,0,0,0,0,0,0])  
  if tag=="greeting":
    output.append([0,0,0,0,0,1,0,0,0,0,0,0])
  if tag=="home page":
    output.append([0,0,0,0,0,0,1,0,0,0,0,0])
  if tag=="options":
    output.append([0,0,0,0,0,0,0,1,0,0,0,0])
  if tag=="train complaints":
    output.append([0,0,0,0,0,0,0,0,1,0,0,0])
  if tag=="train fees":
    output.append([0,0,0,0,0,0,0,0,0,1,0,0])
  if tag=="train ticket booking":
    output.append([0,0,0,0,0,0,0,0,0,0,1,0])
  if tag=="train times":
    output.append([0,0,0,0,0,0,0,0,0,0,0,1])

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [0]:
tfidf_vectorizer = TfidfVectorizer(min_df=0,max_df =0.9,ngram_range=(1,2))    #max_df = 0.9 means "It ignores terms that appear in more than 90% of the documents".   min_df = 2 means "ignore terms that appear in less than 2 documents".
tfidf_vectorizer.fit(X_train)
X_train = tfidf_vectorizer.transform(X_train)

In [0]:
import pickle
with open('/content/drive/My Drive/sem5_group_project/vectorizer.pk', 'wb') as fin:
  pickle.dump(tfidf_vectorizer, fin)

In [0]:
vectorizer_sadil = pickle.load( open( "/content/drive/My Drive/sem5_group_project/vectorizer.pk", "rb" ) )

In [0]:
X_train.shape

(254, 385)

# Model creation


In [0]:
from xgboost import XGBClassifier
model=XGBClassifier()
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [0]:
model.save_model('xgboost_model.json')

In [0]:
import xgboost.sklearn

In [0]:
x="what you can do for me"

X = [text_prepare(x)] 
y=vectorizer_sadil.transform(X)

In [0]:
#y_pred = model.predict_proba(y)
probability = model.predict_proba(y)

In [0]:
probability

array([[0.03396508, 0.00184387, 0.00199562, 0.05493705, 0.06647961,
        0.16432701, 0.32244787, 0.21453613, 0.12899993, 0.00188209,
        0.00232807, 0.00625768]], dtype=float32)

In [0]:
if (max(p[0])>0.5):
  result = np.where(p[0] == max(p[0]))
  print (result)
  print (result[0][0])
  print (labels[result[0][0]])
else:
  print ("sorry! I cannot understand you:(")

sorry! I cannot understand you:(
